In [1]:
# ------------------------------------------------------------------
# Predict the Loan Status using Logistic Regression in scikit-learn
# ------------------------------------------------------------------

# Import required classes from Azureml
from azureml.core import Workspace, Run

# Access the Workspace
ws = Workspace.from_config("./config")

In [2]:
# Get the context of the experiment run
new_run = Run.get_context()


In [3]:
import pandas as pd 
import numpy as np
df = pd.read_csv(r'C:\Users\msyed\OneDrive - Kagool Ltd\Documents\Experiments\Exp1\alldatasets\Loan Approval Prediction.csv')

OSError: [Errno 22] Invalid argument: 'C:\\Users\\msyed\\OneDrive - Kagool Ltd\\Documents\\Experiments\\Exp1\\alldatasets\\Loan Approval Prediction.csv'

In [5]:
# Select columns from the dataset
LoanPrep = df[["Married", 
             "Education",
             "Self_Employed",
             "ApplicantIncome",
             "LoanAmount",
             "Loan_Amount_Term",
             "Credit_History",
             "Loan_Status"
             ]]


In [6]:
# Clean Missing Data - Drop the columns with missing values
LoanPrep = LoanPrep.dropna()


# Create Dummy variables - Not required in designer
LoanPrep = pd.get_dummies(LoanPrep, drop_first=True)

# Create X and Y - Similar to "edit columns" in Train Module
Y = LoanPrep[['Loan_Status_Y']]
X = LoanPrep.drop(['Loan_Status_Y'], axis=1)


In [7]:
# Split Data - X and Y datasets are training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = \
train_test_split(X, Y, test_size = 0.3, random_state = 1234, stratify=Y)


In [8]:
# Build the Logistic Regression model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [9]:

# Fit the data to the LogisticRegression object - Train Model
lr.fit(X_train, Y_train)


# Predict the outcome using Test data - Score Model 
# Scored Label
Y_predict = lr.predict(X_test)

# Get the probability score - Scored Probabilities
Y_prob = lr.predict_proba(X_test)[:, 1]

c:\Users\msyed\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
# Get Confusion matrix and the accuracy/score - Evaluate
from sklearn.metrics import confusion_matrix
cm    = confusion_matrix(Y_test, Y_predict)
score = lr.score(X_test, Y_test)


In [11]:
# -----------------------------------------------------
# Log metrics and Complete an experiment run
# -----------------------------------------------------

# Create the confusion matrix dictionary
cm_dict = {"schema_type": "confusion_matrix",
           "schema_version": "v1",
           "data": {"class_labels": ["N", "Y"],
                    "matrix": cm.tolist()}
           }

new_run.log("TotalObservations", len(df))
new_run.log_confusion_matrix("ConfusionMatrix", cm_dict)
new_run.log("Score", score)


NameError: name 'new_run' is not defined